# Data pre-processing

In [1]:
from src.Utils import *

In [4]:
# Importing training dataset
TRAIN = pd.read_csv(filepath_or_buffer=Utils.FILENAMES["TRAIN"])

In [5]:
# Importing second training dataset
TRAIN_2 = pd.read_csv(filepath_or_buffer=Utils.FILENAMES["TRAIN_2"])

In [ ]:
# Importing test dataset
TEST = pd.read_csv(filepath_or_buffer=Utils.FILENAMESNAMES["TEST"])

After an in-depth visualization, we have observed that the few samples contained inside the `TEST` dataset are

In [6]:
# Verify the continuity of the time series
#TRAIN.tail()
#TRAIN_2.head()

In [9]:
# Merging TRAIN and TRAIN_2 as df
df = pd.concat([TRAIN, TRAIN_2], axis = 0)

**Comments**:
- First timestamp: **2018-01-01 00:01:00**
- Last timestamp: **2022-01-24 00:00:00**

We are now:
1. Splitting our dataframe by `asset_id`
2. Removing unnecessary columns
3. Converting the timestamp as a pandas `DateTime` object
4. Setting this datetime as index
5. Performing some additional operations to compress the data and remove some non-expected precision
6. Saving to disk as CSV and Parquet files for each `asset_id`

For instance, the column `Count` is an integer-value property by definition.
However, our dataset and/or import procedure casted it as a floatting-point number.

$\longrightarrow$ We will reduce this precision using the `.astype()` method.

In [74]:
# Hashmap of train datasets for each asset id
df_dict = {asset_id: None for asset_id in Utils.ASSET_IDS}

# For each asset id, perform the pre-processing
for asset_id in Utils.ASSET_IDS:
    print(f"--> Processing Asset #{asset_id} \t ({Utils.get_asset_name(asset_id)})")
    
    # Retrieving the corresponding data rows
    df_dict[asset_id] = df[df["Asset_ID"] == asset_id]
    
    # Removing the Asset_ID column (useless now)
    try:
        df_dict[asset_id].drop(["Asset_ID", "Target"], axis=1, inplace=True)
    except:
        pass
    
    # Converting the timestamp column
    df_dict[asset_id]["timestamp"] = pd.to_datetime(arg=df_dict[asset_id]["timestamp"], 
                                                     unit="s", 
                                                     errors="ignore")
    # Setting the timestamp column as index column
    df_dict[asset_id].set_index(["timestamp"], inplace=True)
    
    # Converting the Count number
    df_dict[asset_id]["Count"] = pd.to_numeric(arg=df_dict[asset_id]["Count"], 
                                               downcast="integer")

    # Saving it to a new CSV file in assets/
    df_dict[asset_id].to_csv(path_or_buf=f"assets/csv/{asset_id}.csv")
    
    # Saving it to a new Parquet file in assets/ (better for file I/O speed & compression)
    table = pa.Table.from_pandas(df=df_dict[asset_id])
    pq.write_table(table=table, where=f"assets/parquet/{asset_id}.parquet")

--> Processing Asset #0 	 (Binance Coin)
--> Processing Asset #1 	 (Bitcoin)
--> Processing Asset #2 	 (Bitcoin Cash)
--> Processing Asset #3 	 (Cardano)
--> Processing Asset #4 	 (Dogecoin)
--> Processing Asset #5 	 (EOS.IO)
--> Processing Asset #6 	 (Ethereum)
--> Processing Asset #7 	 (Ethereum Classic)
--> Processing Asset #8 	 (IOTA)
--> Processing Asset #9 	 (Litecoin)
--> Processing Asset #10 	 (Maker)
--> Processing Asset #11 	 (Monero)
--> Processing Asset #12 	 (Stellar)
--> Processing Asset #13 	 (TRON)


## Sanity check

We want to check if the sum of the number of rows for each `asset_id` is equal to the number of rows from the original dataset

In [18]:
assert sum([df_dict[k].shape[0] for k in Utils.ASSET_IDS]) == df.shape[0]

We can also check the memory usage of the newly created pandas DataFrames:

In [20]:
# Computing the memory usage of each DataFrame
MEM_USAGE = {asset_id: df_dict[asset_id].memory_usage(index=True).sum()/10**6 for asset_id in Utils.ASSET_IDS}

# Computing the global memory usage
GLOBAL_MEM_USAGE = sum(list(MEM_USAGE.values()))/10**3
print(f"Global mem. usage: \t {GLOBAL_MEM_USAGE :.1f} GB")

Global mem. usage: 	 1.7 GB


In [41]:
pq.read_pandas(source="assets/test.parquet").to_pandas()

,Count,Open,High,Low,Close,Volume,VWAP
timestamp,,,,,,,
2018-01-01 00:01:00,229.0,13835.194000,14013.8,13666.11,13850.176000,31.550062,13827.062093
2018-01-01 00:02:00,235.0,13835.036000,14052.3,13680.00,13828.102000,31.046432,13840.362591
2018-01-01 00:03:00,528.0,13823.900000,14000.4,13601.00,13801.314000,55.061820,13806.068014
2018-01-01 00:04:00,435.0,13802.512000,13999.0,13576.28,13768.040000,38.780529,13783.598101
2018-01-01 00:05:00,742.0,13766.000000,13955.9,13554.44,13724.914000,108.501637,13735.586842
...,...,...,...,...,...,...,...
2022-01-23 23:56:00,1221.0,36278.361667,36331.0,36249.00,36293.123333,30.247067,36289.796806
2022-01-23 23:57:00,1634.0,36289.681667,36328.0,36230.72,36261.708333,35.647000,36276.229399
2022-01-23 23:58:00,1570.0,36262.841667,36319.0,36230.30,36276.223333,49.867700,36274.441548
